# Introduction to Optimization for Decision Making


**Integrantes**:

- **Joel B. Huamani Ccallocunto - 20196510**
- **Luis E. Valverde Ramos - 20191930**


In [1]:
%%html
<iframe src="https://docs.google.com/presentation/d/e/2PACX-1vQHq0p2eTmxRWJjDmo1mUmdarYgIrEew4ieiVbIGQy-D_CyBw5rbbRUlRxwLKKaVQpRV9Hs8MGnz0X2/embed?start=false&loop=false&delayms=3000" frameborder="0" width="960" height="569" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true"></iframe>

# Part 1: Maximization/Minimization

<div class="alert-success">

## <strong>Exercise 1.1: The diet problem</strong>

In [2]:
%%html
<iframe src="https://docs.google.com/presentation/d/e/2PACX-1vTSq9X74urGAB_5n_MIJ9ZGIboKSvBdokVTBXVLh_qqZnmLRTJioOF431Rzys3Qi9UaFwWXjeq6Wmd5/embed?start=false&loop=false&delayms=3000" frameborder="0" width="960" height="569" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true"></iframe>

Please, go to your _environment_ in Anacoda Navigator to install **glpk** and **pulp**  before runing the codes below.
Then, call the library:

In [3]:
#!pip show glpk pulp
#!pip install glpk pulp
#!pip install pulp

Name: PuLP
Version: 2.9.0
Summary: PuLP is an LP modeler written in python. PuLP can generate MPS or LP files and call GLPK, COIN CLP/CBC, CPLEX, and GUROBI to solve linear problems.
Home-page: https://github.com/coin-or/pulp
Author: J.S. Roy and S.A. Mitchell and F. Peschiera
Author-email: pulp@stuartmitchell.com
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: 
Required-by: 


In [4]:
import pulp as pp

1. **Initialize the MODEL**: just write the name and declare if it is maximization or minimization problem type.

In [5]:
model = pp.LpProblem(name='diet-problem', # just the name
                     sense=pp.LpMinimize) # type of problem

2. **Declare the VARIABLES**: The refinery model consists of these _variables_:

In [6]:
VegaVita = pp.LpVariable(name="VegaVita", lowBound=0, cat='Continuous')

HappyHealth = pp.LpVariable(name="HappyHealth", lowBound=0, cat='Continuous')

In [7]:
VegaVitaCost = 0.2
HappyHealthCost = 0.3
# Función objetivo: minimizar el costo
model += VegaVitaCost * VegaVita + HappyHealthCost * HappyHealth, "TotalCost"

3. **Create function to OPTIMIZE**: The function is just the linear combination of the variables and their _given coefficients__:

In [8]:
# Restricciones de requerimientos nutricionales
model += 20 * VegaVita + 30 * HappyHealth >= 60, "Vitamin_C_Requirement"
model += 500 * VegaVita + 250 * HappyHealth >= 1000, "Calcium_Requirement"
model += 9 * VegaVita + 2 * HappyHealth >= 18, "Iron_Requirement"
model += 2 * VegaVita + 10 * HappyHealth >= 20, "Niacin_Requirement"
model += 60 * VegaVita + 90 * HappyHealth >= 360, "Magnesium_Requirement"

In [9]:
#solverToUse = pp.COIN_CMD(msg=False)
model.solve();

In [10]:
import pandas as pd

Results = {
    "Model Status": pp.LpStatus[model.status],
    "Optimal Solution": pp.value(model.objective),
    "VegaVita": VegaVita.varValue,
    "HappyHealth": HappyHealth.varValue
}

In [11]:
df_results = pd.DataFrame.from_dict(Results, orient='index').T.set_index('Model Status')
df_results.style.format('{:.2f}')

,Optimal Solution,VegaVita,HappyHealth
Model Status,,,
Optimal,1.20,1.30,3.13


### Interpretación de los resultados:

El problema planteado es un **problema de dieta** en el que se busca minimizar el costo total de dos alimentos, **VegaVita** y **HappyHealth**, mientras se cumplen ciertos requisitos nutricionales relacionados con vitaminas, calcio, hierro, niacina y magnesio.

Con los resultados obtenidos del modelo de optimización, tenemos lo siguiente:

- **Solución Óptima**: El costo total de la dieta óptima, es decir, la mínima cantidad de dinero necesario para satisfacer todos los requerimientos nutricionales, es **3.13 unidades** (el valor exacto depende de las unidades del problema, pero aquí se asume que es el costo mínimo basado en las variables de las decisiones, VegaVita y HappyHealth).
  
- **VegaVita**: Para alcanzar la solución óptima, se debe consumir **1.20 unidades** de **VegaVita**.
  
- **HappyHealth**: Para alcanzar la solución óptima, se debe consumir **1.30 unidades** de **HappyHealth**.

### Conclusiones:

- Para **satisfacer todos los requerimientos nutricionales** de la dieta de manera óptima, el modelo sugiere consumir **1.20 unidades de VegaVita** y **1.30 unidades de HappyHealth**.
- El costo total mínimo de esta combinación es **3.13 unidades**.

Este tipo de análisis es útil para la planificación de dietas balanceadas donde el objetivo es optimizar costos manteniendo la nutrición necesaria.

<div class="alert-success">

## <strong>Exercise 1.2: The scheduling problem</strong>

In [53]:
%%html
<iframe src="https://docs.google.com/presentation/d/e/2PACX-1vQtBRpIr6Hx1_T0zJ3_DRqsE82YUjx7ZkeEKLdA64fbjtjkmc6Ibf6ebzp6CY69D482IGpG2h9GcsC5/embed?start=false&loop=false&delayms=3000" frameborder="0" width="960" height="569" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true"></iframe>

In [71]:
# Importar la biblioteca PuLP
import pulp as pp
import pandas as pd

In [72]:
# Crear el modelo de optimización
model = pp.LpProblem(name='scheduling-problem', sense=pp.LpMinimize)

In [73]:
# Variables de decisión: cantidad de conductores en cada turno
DR_0_8 = pp.LpVariable(name="DR_0_8", lowBound=0, cat="Integer")
DR_4_12 = pp.LpVariable(name="DR_4_12", lowBound=0, cat="Integer")
DR_8_16 = pp.LpVariable(name="DR_8_16", lowBound=0, cat="Integer")
DR_12_20 = pp.LpVariable(name="DR_12_20", lowBound=0, cat="Integer")
DR_16_0 = pp.LpVariable(name="DR_16_0", lowBound=0, cat="Integer")
DR_20_4 = pp.LpVariable(name="DR_20_4", lowBound=0, cat="Integer")

In [75]:
# Función objetivo: minimizar el número total de conductores
model += (
    DR_0_8 + DR_4_12 + DR_8_16 + DR_12_20 + DR_16_0 + DR_20_4,
    "Total Drivers"
)

In [76]:
# Restricciones de demanda por cada franja horaria
model += (DR_0_8 + DR_20_4 >= 4, "Demand_0_4")
model += (DR_0_8 + DR_4_12 >= 8, "Demand_4_8")
model += (DR_4_12 + DR_8_16 >= 10, "Demand_8_12")
model += (DR_8_16 + DR_12_20 >= 7, "Demand_12_16")
model += (DR_12_20 + DR_16_0 >= 12, "Demand_16_20")
model += (DR_16_0 + DR_20_4 >= 4, "Demand_20_0")

In [77]:
# Resolver el modelo
solver = pp.PULP_CBC_CMD()  # Solver por defecto de PuLP
model.solve(solver)

1

In [78]:
# Extraer resultados
results = {
    "Model Status": pp.LpStatus[model.status],
    "Objective Value": pp.value(model.objective),
    "DR_0_8": DR_0_8.varValue,
    "DR_4_12": DR_4_12.varValue,
    "DR_8_16": DR_8_16.varValue,
    "DR_12_20": DR_12_20.varValue,
    "DR_16_0": DR_16_0.varValue,
    "DR_20_4": DR_20_4.varValue,
}

In [79]:
# Crear el DataFrame sin formateo adicional
results_df = pd.DataFrame.from_dict(results, orient="index", columns=["Value"])
print(results_df)

                   Value
Model Status     Optimal
Objective Value     26.0
DR_0_8               0.0
DR_4_12             10.0
DR_8_16              0.0
DR_12_20            12.0
DR_16_0              0.0
DR_20_4              4.0


Los resultados indican la solución óptima del problema de programación para los conductores de autobuses, interpretada de la siguiente manera:

### Estado del modelo
- **Model Status**: *Optimal*  
  Esto significa que se encontró una solución que satisface todas las restricciones y minimiza el número total de conductores contratados.

### Valor objetivo
- **Objective Value**: *26.0*  
  El número total mínimo de conductores necesarios para cubrir la demanda de transporte es **26**.

### Asignación de conductores por turno
1. **DR_0_8**: *0.0*  
   Ningún conductor debe ser asignado al turno de 0:00 a 8:00.

2. **DR_4_12**: *10.0*  
   Se necesitan **10 conductores** para cubrir el turno de 4:00 a 12:00.

3. **DR_8_16**: *0.0*  
   Ningún conductor debe ser asignado al turno de 8:00 a 16:00.

4. **DR_12_20**: *12.0*  
   Se necesitan **12 conductores** para cubrir el turno de 12:00 a 20:00.

5. **DR_16_0**: *0.0*  
   Ningún conductor debe ser asignado al turno de 16:00 a 0:00.

6. **DR_20_4**: *4.0*  
   Se necesitan **4 conductores** para cubrir el turno de 20:00 a 4:00.

### Conclusión
- La demanda histórica de transporte se satisface con la contratación mínima de **26 conductores**.
- No se asignan conductores a los turnos de 0:00-8:00, 8:00-16:00 y 16:00-0:00, lo que optimiza la programación al enfocarse en los turnos con mayor demanda: 4:00-12:00, 12:00-20:00 y 20:00-4:00.
  
Esta solución reduce costos y asegura que haya suficientes conductores para los periodos de mayor demanda, respetando las restricciones del problema.

# Part 2: Multicriteria Decision-Making

In [54]:
%%html
<iframe src="https://docs.google.com/presentation/d/e/2PACX-1vR7GL_wF1eKRO0JgEUyIx5cxXUhTQ8ZM4F3AE1MLr7GYG33dwEobrLo6O2MaV2d7Cv47TaTgHghkhrV/embed?start=false&loop=false&delayms=3000" frameborder="0" width="960" height="569" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true"></iframe>

1. Prepare data file with the comparissons:

In [12]:
%%html

<iframe src="https://docs.google.com/spreadsheets/d/e/2PACX-1vSeUfh-DtfAAvEecirNS7Qs2qN4npmNfRiw9JvKmRpq88snVc8HJBlru2cyPy8lsQflSxlnx6U-IePw/pubhtml?widget=true&amp;headers=false" width="600" height="300" ></iframe>

<div class="alert-success">

## <strong>Exercise 2.1: Choosing a country for a Master Program</strong>

- Make a group of 4 people from this course.
- If you have the criteria: cost of living, language difficulty, possibilities to get a job in that country after studies are finished.
- If you have the alternatives: Brazil, Spain, USA, Germany.
- Create an AHP model and get the ranking.

You can follow this [example](https://en.wikipedia.org/wiki/Analytic_hierarchy_process_%E2%80%93_leader_example).
If you have a better idea, you can use it instead.

2. Get the data (Excel)

In [92]:
# the link to the data
linkGithub = 'https://github.com/luccemhu/5-TAREA-1INT46-PUCP/raw/refs/heads/main/comparissons.xlsx'  # the link to the data

3. Open each sheet:

In [93]:
# opening the comparisons
import pandas as pd

# Load comparison data for each criterion from the Google Sheets repository
pairwise_costo_vida = pd.read_excel(linkGithub, sheet_name='Costo', index_col=0)
pairwise_idioma = pd.read_excel(linkGithub, sheet_name='Idioma', index_col=0)
pairwise_empleo = pd.read_excel(linkGithub, sheet_name='Empleo', index_col=0)
pairwise_criteria = pd.read_excel(linkGithub, sheet_name='Criteria', index_col=0)

You may want to check the structure:

In [94]:
pairwise_criteria

,Costo,Idioma,Empleo
Costo,1.0,3.0,5
Idioma,333.0,1.0,2
Empleo,0.2,0.5,1


4. Transform all matrices into pairwise comparissons:

In [95]:
import networkx as nx

G_costo_vida = nx.from_pandas_adjacency(pairwise_costo_vida, create_using=nx.MultiDiGraph())

# pairwise
G_costo_vida.edges(data=True)

OutMultiEdgeDataView([('Brasil', 'Brasil', {'weight': 1.0}), ('Brasil', 'España', {'weight': 3.0}), ('Brasil', 'USA', {'weight': 5.0}), ('Brasil', 'Alemania', {'weight': 7.0}), ('España', 'Brasil', {'weight': 333.0}), ('España', 'España', {'weight': 1.0}), ('España', 'USA', {'weight': 3.0}), ('España', 'Alemania', {'weight': 5.0}), ('USA', 'Brasil', {'weight': 0.2}), ('USA', 'España', {'weight': 333.0}), ('USA', 'USA', {'weight': 1.0}), ('USA', 'Alemania', {'weight': 2.0}), ('Alemania', 'Brasil', {'weight': 142.0}), ('Alemania', 'España', {'weight': 0.2}), ('Alemania', 'USA', {'weight': 0.5}), ('Alemania', 'Alemania', {'weight': 1.0})])

In [96]:
# Extract pairwise comparisons for each criterion
costo_vida_comparisons = {(e[0], e[1]): e[2]['weight'] for e in G_costo_vida.edges(data=True) if e[0] != e[1]}
costo_vida_comparisons

{('Brasil', 'España'): 3.0,
 ('Brasil', 'USA'): 5.0,
 ('Brasil', 'Alemania'): 7.0,
 ('España', 'Brasil'): 333.0,
 ('España', 'USA'): 3.0,
 ('España', 'Alemania'): 5.0,
 ('USA', 'Brasil'): 0.2,
 ('USA', 'España'): 333.0,
 ('USA', 'Alemania'): 2.0,
 ('Alemania', 'Brasil'): 142.0,
 ('Alemania', 'España'): 0.2,
 ('Alemania', 'USA'): 0.5}

In [97]:
# the remaining comparissons:

G_idioma = nx.from_pandas_adjacency(pairwise_idioma, create_using=nx.MultiDiGraph())
idioma_comparisons = {(e[0], e[1]): e[2]['weight'] for e in G_idioma.edges(data=True) if e[0] != e[1]}

G_empleo = nx.from_pandas_adjacency(pairwise_empleo, create_using=nx.MultiDiGraph())
empleo_comparisons = {(e[0], e[1]): e[2]['weight'] for e in G_empleo.edges(data=True) if e[0] != e[1]}


In [98]:
# take a look
[costo_vida_comparisons, idioma_comparisons, empleo_comparisons]


[{('Brasil', 'España'): 3.0,
  ('Brasil', 'USA'): 5.0,
  ('Brasil', 'Alemania'): 7.0,
  ('España', 'Brasil'): 333.0,
  ('España', 'USA'): 3.0,
  ('España', 'Alemania'): 5.0,
  ('USA', 'Brasil'): 0.2,
  ('USA', 'España'): 333.0,
  ('USA', 'Alemania'): 2.0,
  ('Alemania', 'Brasil'): 142.0,
  ('Alemania', 'España'): 0.2,
  ('Alemania', 'USA'): 0.5},
 {('Brasil', 'España'): 5.0,
  ('Brasil', 'USA'): 7.0,
  ('Brasil', 'Alemania'): 9.0,
  ('España', 'Brasil'): 0.2,
  ('España', 'USA'): 3.0,
  ('España', 'Alemania'): 5.0,
  ('USA', 'Brasil'): 142.0,
  ('USA', 'España'): 333.0,
  ('USA', 'Alemania'): 2.0,
  ('Alemania', 'Brasil'): 111.0,
  ('Alemania', 'España'): 0.2,
  ('Alemania', 'USA'): 0.5},
 {('Brasil', 'España'): 0.25,
  ('Brasil', 'USA'): 0.2,
  ('Brasil', 'Alemania'): 142.0,
  ('España', 'Brasil'): 4.0,
  ('España', 'USA'): 333.0,
  ('España', 'Alemania'): 0.2,
  ('USA', 'Brasil'): 5.0,
  ('USA', 'España'): 3.0,
  ('USA', 'Alemania'): 333.0,
  ('Alemania', 'Brasil'): 7.0,
  ('Alemania

In [99]:
# now the criteria

G_CRIT = nx.from_pandas_adjacency(pairwise_criteria, create_using=nx.MultiDiGraph())
criteria_comparisons = {(e[0], e[1]): e[2]['weight'] for e in G_CRIT.edges(data=True) if e[0] != e[1]}
criteria_comparisons

{('Costo', 'Idioma'): 3.0,
 ('Costo', 'Empleo'): 5.0,
 ('Idioma', 'Costo'): 333.0,
 ('Idioma', 'Empleo'): 2.0,
 ('Empleo', 'Costo'): 0.2,
 ('Empleo', 'Idioma'): 0.5}

5. Apply the Algorithm

In [100]:
## install
#!pip install ahpy

In [101]:
# input each comparisson

import ahpy

# Creating comparison models for each criterion
costo_vida = ahpy.Compare('costo_vida', costo_vida_comparisons, precision=3, random_index='saaty')
idioma = ahpy.Compare('idioma', idioma_comparisons, precision=3, random_index='saaty')
empleo = ahpy.Compare('empleo', empleo_comparisons, precision=3, random_index='saaty')

# Create the parent criteria model (for ranking the 3 criteria)
criteria = ahpy.Compare('criteria', criteria_comparisons, precision=3, random_index='saaty')

/usr/local/lib/python3.10/dist-packages/numpy/linalg/linalg.py:667: RuntimeWarning: overflow encountered in matmul
  return fmatmul(a, a)
/usr/local/lib/python3.10/dist-packages/ahpy/ahpy.py:253: RuntimeWarning: invalid value encountered in divide
  principal_eigenvector = np.divide(row_sum, total_sum)


6. Create hierarchy:

In [102]:
criteria.add_children([costo_vida, idioma, empleo])

7. See result:

In [103]:
print(criteria.target_weights)

{}


8. Assess consistency

In [106]:
## We should review comparissons if greater than 0.1!
[(val.name, val.consistency_ratio) for val in [costo_vida, idioma, empleo, criteria]]


[('costo_vida', 29.957),
 ('idioma', 4.773),
 ('empleo', 0.067),
 ('criteria', 7.225)]

### Interpretación de los resultados

1. **Costo de vida (29.957)**:
   - Este valor es significativamente alto, lo que indica que las comparaciones realizadas para este criterio no son del todo consistentes. El índice de consistencia en AHP debe ser inferior a 10 para considerarse razonable, por lo que un valor de 29.957 es un claro indicativo de que las comparaciones realizadas para el criterio de "Costo de vida" pueden no ser lo suficientemente consistentes. Esto sugiere que puede haber ambigüedad o inconsistencias en las percepciones de la importancia relativa de este criterio en comparación con los demás, lo que podría deberse a diferencias subjetivas en las evaluaciones o a posibles errores en la asignación de los pesos de las alternativas.

2. **Idioma (4.773)**:
   - Aunque el valor es más bajo que el de "Costo de vida", sigue siendo algo elevado. Un índice de consistencia de 4.773 también sugiere que las comparaciones para este criterio tienen cierto grado de inconsistencia, aunque en menor medida que las de "Costo de vida". Sin embargo, aún está dentro del rango tolerable si consideramos que los valores generalmente aceptados en AHP deberían estar por debajo de 10. Esta inconsistencia podría reflejar las dificultades inherentes al evaluar un criterio como la "dificultad del idioma", que depende de muchos factores subjetivos y contextuales, como las habilidades previas de los participantes o el nivel de familiaridad con los idiomas de los países evaluados.

3. **Empleo (0.067)**:
   - Este valor es extremadamente bajo y está muy por debajo de 10, lo cual indica una alta consistencia en las comparaciones realizadas para el criterio de "Posibilidades de conseguir empleo". Esto sugiere que las evaluaciones para este criterio son mucho más consistentes que las de los otros dos. Es posible que el factor "empleo" sea más claro o menos ambiguo para los participantes, dado que tiene una relación más directa con datos objetivos, como tasas de desempleo o políticas laborales de los países evaluados.

4. **Criteria (7.225)**:
   - El valor para el índice de consistencia general de las comparaciones de los criterios también es relativamente bajo, lo que indica que el proceso de comparación entre los tres criterios ("Costo de vida", "Idioma" y "Empleo") fue relativamente consistente. Este valor sugiere que las percepciones de la importancia relativa de los tres criterios no son totalmente incoherentes, aunque podría mejorarse la consistencia.

### Conclusión:
El análisis de consistencia de las comparaciones en este modelo AHP revela que las comparaciones para el criterio de "Costo de vida" son las más inconsistentes, lo que podría señalar la necesidad de una revisión más profunda o más precisa de este criterio. En contraste, las comparaciones para el criterio de "Empleo" muestran una alta consistencia, lo cual es un buen indicador de que este criterio está siendo evaluado de manera clara y coherente. En general, el índice de consistencia para el modelo completo está dentro de un rango aceptable, pero se debe tener en cuenta que las inconsistencias en algunos de los criterios pueden afectar la precisión del modelo AHP y, por lo tanto, las decisiones derivadas del mismo.

Es recomendable revisar las comparaciones y, si es posible, ajustar o clarificar las percepciones de los participantes para mejorar la consistencia en futuros análisis.

Nota: Los resultados reflejan los **índices de consistencia** de las comparaciones realizadas en el proceso AHP para los diferentes criterios. En AHP, el **índice de consistencia** (o **ratio de consistencia**) mide cuán coherentes son las comparaciones entre los elementos. Un valor cercano a 0 indica que las comparaciones son consistentes, mientras que un valor más alto sugiere que las comparaciones pueden ser inconsistentes.

# Part 3: Benchmarking

<div class="alert-success">

<strong>Exercise: Efficiency in Public sector</strong>

- Make a group of 2 people from this course.
- Find a set of municipalities (homogenity required).
- Find a set of common input and out variables for them.
- Compute efficiency.

In [16]:
import pandas as pd

# Specify the encoding when reading the CSV file
renamu = pd.read_csv("https://github.com/luccemhu/5-TAREA-1INT46-PUCP/raw/refs/heads/main/renamu_data.csv", encoding='latin-1')
# or 'ISO-8859-1' or the correct encoding

print(renamu)

                    Distrito  Ingresos_Total  Gastos_Total  Personal_Limpieza  \
0                CHACHAPOYAS     31477166.86   29581273.12                 24   
1                   ASUNCION      1265551.60     976582.75                  0   
2                     BALSAS      1376502.78    1303334.70                  1   
3                      CHETO      2083770.19    2132187.20                  0   
4                  CHILIQUIN      1347598.51    1380533.45                  0   
...                      ...             ...           ...                ...   
1869                 IRAZOLA      9149578.11    8705528.29                  0   
1870                CURIMANA     13139854.58   12214407.65                  2   
1871                 NESHUYA      9829463.30    9497257.41                  0   
1872  ALEXANDER VON HUMBOLDT      4534922.58    3911376.27                  0   
1873                   PURUS      8185867.56    7116701.83                  7   

      Residuos_Solidos  Per

In [17]:
renamu.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1874 entries, 0 to 1873
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Distrito             1874 non-null   object 
 1   Ingresos_Total       1874 non-null   float64
 2   Gastos_Total         1874 non-null   float64
 3   Personal_Limpieza    1874 non-null   int64  
 4   Residuos_Solidos     1874 non-null   float64
 5   Personal_Vigilancia  1874 non-null   int64  
dtypes: float64(3), int64(2), object(1)
memory usage: 88.0+ KB


The first three variables (Aircraft,Fuel,Employee) represent **inputs** and the last two ones represent **outputs**. If that is so, there should be a way to compute some measure of efficiency: the ratio **output/input**.

Let's compute some ratios:

In [18]:
# ratio passenger employee:
renamu['rate_GastosByIngresos']=(renamu.Gastos_Total/renamu.Ingresos_Total)
renamu['rate_ResiduosByPersonal']=(renamu.Residuos_Solidos/renamu.Personal_Limpieza)

In [21]:
import numpy as np

# Reemplazar los valores inf por NaN
renamu['rate_GastosByIngresos'] = renamu['rate_GastosByIngresos'].replace([np.inf, -np.inf], np.nan)
renamu['rate_ResiduosByPersonal'] = renamu['rate_ResiduosByPersonal'].replace([np.inf, -np.inf], np.nan)
renamu = renamu.dropna(subset=['rate_GastosByIngresos', 'rate_ResiduosByPersonal'])
renamu.info()


<class 'pandas.core.frame.DataFrame'>
Index: 864 entries, 0 to 1873
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Distrito                 864 non-null    object 
 1   Ingresos_Total           864 non-null    float64
 2   Gastos_Total             864 non-null    float64
 3   Personal_Limpieza        864 non-null    int64  
 4   Residuos_Solidos         864 non-null    float64
 5   Personal_Vigilancia      864 non-null    int64  
 6   rate_GastosByIngresos    864 non-null    float64
 7   rate_ResiduosByPersonal  864 non-null    float64
dtypes: float64(5), int64(2), object(1)
memory usage: 60.8+ KB


Let me plot those ratios:

In [24]:
import altair as alt

# Gráfico de puntos con tamaño ajustado
points = alt.Chart(renamu).mark_point().encode(
    x='rate_GastosByIngresos:Q',
    y=alt.Y('rate_ResiduosByPersonal:Q', scale=alt.Scale(type='log')),  # Escala logarítmica en el eje Y
).properties(
    width=800,  # Ajusta el ancho del gráfico
    height=600  # Ajusta la altura del gráfico
)

# Texto con tamaño de fuente reducido
text = points.mark_text(
    align='right',
    baseline='middle',
    dx=-7,
    fontSize=8  # Ajusta el tamaño de la fuente aquí
).encode(
    text='Distrito:N'
).interactive()

# Se combina el gráfico de puntos y el texto
points + text


/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.LayerChart(...)

Which one is more efficient? As you see, one airline might not be good in both ratios:

In [25]:
renamu[['Distrito','rate_GastosByIngresos','rate_ResiduosByPersonal']].sort_values(by='rate_GastosByIngresos',ascending=False).head()

,Distrito,rate_GastosByIngresos,rate_ResiduosByPersonal
75,TOTORA,2.422192,80.000000
667,LLAPA,2.162925,33.333333
561,VISCHONGO,1.970919,400.000000
35,SAN CARLOS,1.863380,300.000000
1479,CAPELO,1.619560,45.000000


In [26]:
renamu[['Distrito','rate_GastosByIngresos','rate_ResiduosByPersonal']].sort_values(by='rate_ResiduosByPersonal',ascending=False).head()

,Distrito,rate_GastosByIngresos,rate_ResiduosByPersonal
1295,LINCE,0.809568,66789.000000
1293,LA MOLINA,0.968355,22571.428571
1618,EL ALTO,0.778061,16000.000000
1020,MARCONA,0.469297,14730.000000
702,SAYLLA,0.676013,13900.000000


Let me show you the **envelope**:

In [27]:
Best_GastosByIngresos=renamu.rate_GastosByIngresos.idxmax()
Best_ResiduosByPersonal=renamu.rate_ResiduosByPersonal.idxmax()

frontier1=renamu.loc[Best_GastosByIngresos,['rate_GastosByIngresos','rate_ResiduosByPersonal']].to_list()
frontier2=renamu.loc[Best_ResiduosByPersonal,['rate_GastosByIngresos','rate_ResiduosByPersonal']].to_list()

# Fronteras paralelas
frontier1v = [frontier1[0], 0]
frontier2h = [0, frontier2[1]]

#then
envelope = pd.DataFrame([frontier2h, frontier2, frontier1, frontier1v], columns=['x', 'y'])
envelope

,x,y
0,0.000000,66789.0
1,0.809568,66789.0
2,2.422192,80.0
3,2.422192,0.0


Updating the plot:

In [33]:
points + text + alt.Chart(envelope).mark_line(color='red').encode(
    x='x',
    y='y',
)

/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.LayerChart(...)

The presence of several units (DMUs), several inputs, and several outputs makes it difficult to judge who is doing better. This an optimization problem that may be carried out using **Pyfrontier**:

In [29]:
## installation
#!pip install Pyfrontier

Let me recover the names for each group of variables:

In [30]:
renamu.info()

<class 'pandas.core.frame.DataFrame'>
Index: 864 entries, 0 to 1873
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Distrito                 864 non-null    object 
 1   Ingresos_Total           864 non-null    float64
 2   Gastos_Total             864 non-null    float64
 3   Personal_Limpieza        864 non-null    int64  
 4   Residuos_Solidos         864 non-null    float64
 5   Personal_Vigilancia      864 non-null    int64  
 6   rate_GastosByIngresos    864 non-null    float64
 7   rate_ResiduosByPersonal  864 non-null    float64
dtypes: float64(5), int64(2), object(1)
memory usage: 93.0+ KB


In [31]:
renamuInput = renamu.columns[[1, 3]].to_list()  # Use a list for indexing
renamuOutput = renamu.columns[[2, 4]].to_list()  # Use a list for indexing

Let's apply the function:

In [32]:
from Pyfrontier.frontier_model import EnvelopDEA

dea_air_vrs_in = EnvelopDEA("VRS", "in")
dea_air_vrs_in.fit(
    inputs=renamu[renamuInput].to_numpy(),
    outputs=renamu[renamuOutput].to_numpy()
)

Here is the result:

In [35]:
renamu['vrs_in']=[r.score for r in dea_air_vrs_in.result]
renamu.set_index(renamu.Distrito,inplace=True)
renamu['vrs_in']

,vrs_in
Distrito,
CHACHAPOYAS,0.645615
BALSAS,1.000000
MARISCAL CASTILLA,1.000000
MONTEVIDEO,1.000000
SOLOCO,1.000000
...,...
MANANTAY,0.776966
RAIMONDI,0.508769
SEPAHUA,1.000000


At this stage, you may be tempted to do a regression:

In [36]:
#!pip install py4etrics

In [37]:
import numpy as np # linear algebra
from py4etrics import tobit
# import statsmodels.api as sm

renamu['censored'] =np.where(renamu['vrs_in']==1, 1, 0)
cens = renamu['censored']
endog = renamu.loc[:,'vrs_in']
exog = renamu.loc[:,'Ingresos_Total':'Residuos_Solidos']

tobit_res = tobit.Tobit(endog, exog, cens,right=1).fit()
tobit_res.summary()

Optimization terminated successfully.
         Current function value: 1.010423
         Iterations: 252
         Function evaluations: 389


/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:595: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '


<class 'statsmodels.iolib.summary.Summary'>
"""
                              Tobit Regression Results                             
===================================================================================
Dep. Variable:                      vrs_in   Pseudo R-squ:                   -1.593
Method:                 Maximum Likelihood   Log-Likelihood:                 -873.0
No. Observations:                      864   LL-Null:                        -336.7
No. Uncensored Obs:                    674   LL-Ratio:                      -1072.6
No. Left-censored Obs:                   0   LLR p-value:                     1.000
No. Right-censored Obs:                190   AIC:                            1752.0
Df Residuals:                          860   BIC:                            1766.3
Df Model:                                3   Covariance Type:             nonrobust
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Ingresos_Total    -6.707e-09        nan        nan        nan         nan         nan
Gastos_Total       2.392e-08        nan        nan        nan         nan         nan
Personal_Limpieza    -0.0021        nan        nan        nan         nan         nan
Residuos_Solidos  -6.602e-07        nan        nan        nan         nan         nan
Log(Sigma)           -0.4469        nan        nan        nan         nan         nan
=====================================================================================
"""


### 1. **Resumen General del Modelo:**
   - **Dependiente (Variable endógena):** La variable dependiente es `vrs_in`, que parece ser una medida de eficiencia en los municipios. Esta variable se obtiene a través de un análisis de envolvente de datos (DEA) utilizando un enfoque de "Variable Returns to Scale" (VRS), donde se mide la eficiencia técnica de los municipios en términos de la relación entre sus inputs (por ejemplo, ingresos y personal) y outputs (por ejemplo, residuos sólidos, gastos).
   - **Número de Observaciones:** El modelo se ha ajustado a 864 observaciones, de las cuales 674 no están censuradas (es decir, tienen valores observados) y 190 están censuradas a la derecha (es decir, no se sabe el valor exacto, pero se sabe que está por encima de un cierto umbral).
   - **Pseudo R-cuadrado:** El valor negativo de -1.593 sugiere que el modelo no explica bien la variabilidad de la variable dependiente (algo común en modelos Tobit, ya que el objetivo no es predecir un valor exacto sino estimar la probabilidad de que la variable dependiente esté censurada).
   - **Log-Likelihood:** Este valor es -873, que indica la bondad de ajuste del modelo. Cuanto más cerca de cero sea el log-likelihood, mejor es el ajuste del modelo.
   - **AIC (Criterio de Información de Akaike):** 1752.0. Este es un indicador del ajuste del modelo penalizado por la complejidad (número de parámetros estimados). Un AIC más bajo indica un mejor modelo ajustado.
   - **BIC (Criterio de Información Bayesiano):** 1766.3, similar al AIC, pero penaliza más los modelos con más parámetros. Un valor más bajo de BIC también indica mejor ajuste.

### 2. **Coeficientes de la Regresión Tobit:**
   Los coeficientes estimados para cada variable explicativa indican cómo afectan las variables predictoras a la variable dependiente `vrs_in`. En este caso, los coeficientes tienen una magnitud muy pequeña, lo que sugiere que el impacto de cada una de las variables independientes sobre la eficiencia es bajo. Sin embargo, el error estándar y la significancia de los coeficientes no están disponibles, lo que indica problemas con la convergencia del modelo.

   - **`Ingresos_Total`:** El coeficiente estimado es **-6.707e-09**. Este valor es extremadamente pequeño, lo que sugiere que los ingresos totales no tienen un impacto significativo en la eficiencia medida por `vrs_in` (probablemente debido a una alta multicolinealidad o falta de variabilidad en la variable).
   - **`Gastos_Total`:** El coeficiente estimado es **2.392e-08**. Aunque es pequeño, indica una relación positiva con la eficiencia, lo que sugiere que un aumento en los gastos podría estar asociado a un pequeño aumento en la eficiencia de los municipios. Al igual que en el caso de los ingresos, la falta de significancia y la magnitud pequeña podrían ser indicativos de un problema en el ajuste del modelo.
   - **`Personal_Limpieza`:** El coeficiente estimado es **-0.0021**, lo que indica que un aumento en el personal de limpieza está asociado a una disminución en la eficiencia de los municipios. Sin embargo, el hecho de que el error estándar no esté disponible sugiere que esta estimación podría no ser fiable.
   - **`Residuos_Solidos`:** El coeficiente estimado es **-6.602e-07**, lo que sugiere que un aumento en los residuos sólidos está asociado a una disminución en la eficiencia, pero nuevamente, la falta de significancia y la magnitud pequeña podrían cuestionar la validez de este resultado.

### 3. **Problemas con la Convergencia:**
   - El **`HessianInversionWarning`** indica que hubo un problema al intentar invertir la matriz Hessiana, que es una parte esencial del cálculo de los errores estándar y la covarianza de los coeficientes. Esto sugiere que el modelo podría no haber convergido correctamente, lo que puede ser el resultado de multicolinealidad entre las variables predictoras, una distribución sesgada de los datos, o un modelo mal especificado.
   
   - Además, la presencia de **"nan" (Not a Number)** en los errores estándar sugiere que el modelo no ha podido estimar correctamente los parámetros de manera fiable, lo que podría deberse a la colinealidad entre las variables, la censura en los datos o problemas en la estructura del modelo.

### 4. **Variables Censuradas:**
   - La **variable censurada** se ha creado para indicar si la variable dependiente (`vrs_in`) es igual a 1, que indica que la observación está en el límite superior de la eficiencia. El modelo Tobit se utiliza precisamente para lidiar con estas observaciones censuradas, por lo que la censura es un aspecto importante a considerar en la interpretación de los resultados.

### Conclusión:
El modelo Tobit que has ajustado no parece ofrecer resultados claros y fiables debido a problemas con la convergencia y la falta de significancia de los coeficientes. La interpretación de los coeficientes debe tomarse con cautela debido a los problemas de estimación, y podría ser útil revisar los datos (por ejemplo, comprobando la multicolinealidad entre variables), ajustar el modelo o explorar otras metodologías de análisis como el modelo de fronteras eficientes o regresión con otro tipo de restricciones.

Además, sería importante examinar los errores y el ajuste del modelo en más detalle para determinar si hay alguna especificación incorrecta o un problema con la selección de las variables independientes.


Nota: Los resultados del modelo de regresión Tobit proporcionan un análisis de la eficiencia en el sector público, específicamente, de los municipios, utilizando las variables seleccionadas para el análisis.

